<h1>Pipeline Deployment</h1>

Once we have trained our models for feature engineering and binary classification, we can choose to deploy them in pipeline using Amazon SageMaker Inference Pipelines.

This notebook demonstrates how to create a pipeline with the SKLearn model for feature engineering and the Amazon SageMaker Linear Learner model for binary classification.

In [26]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session(boto3.Session())

print(sagemaker_session)

print(region)
print(role)

eu-west-1
arn:aws:iam::825935527263:role/gianpo-path/SageMaker-Notebook-Role


First, we need to create SageMaker Model objects, which represent the objects that binds together the artifacts of training (model artifacts as S3 objects) and the Docker container used for inference.

We have to specify the paths to our models in S3:

In [27]:
sklearn_preprocessor_path = 's3://gianpo-predictive-maintenance/output/predmain-expprep-sklearn-2019-05-02-18-50-10-249/output/model.tar.gz'
linear_learner_model_path = 's3://gianpo-predictive-maintenance/output/predmain-training-ll-2019-05-04-14-18-50-815/output/model.tar.gz'


Then we need to find the linear learner Docker containers path in ECR:

In [28]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri

linear_learner_container = get_image_uri(boto3.Session().region_name, 'linear-learner', repo_version="latest")
print(linear_learner_container)

438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest


We are ready to create our models:

In [29]:
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel

sklearn_preprocessor = SKLearnModel(sklearn_preprocessor_path, role, '1-predmain-expprep-sklearn-script.py', sagemaker_session=sagemaker_session)
linear_learner_model = Model(linear_learner_model_path, linear_learner_container, sagemaker_session=sagemaker_session)

Once we have models ready, we can deploy them in a pipeline:

In [30]:
import sagemaker
from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name='predmain-sk-ll-pipeline', 
    role=role, 
    models=[
        sklearn_preprocessor, 
        linear_learner_model],
    sagemaker_session=sagemaker_session)

pipeline_model.deploy(initial_instance_count=1, 
                      instance_type='ml.c5.xlarge', 
                      endpoint_name='predmain-sk-ll-endpoint')

INFO:sagemaker:Creating model with name: predmain-sk-ll-pipeline
INFO:sagemaker:Creating endpoint with name predmain-sk-ll-endpoint


----------------------------------------------------------------------------!

<h2>Getting inferences</h2>

Now we can try invoking our pipeline of models and try getting some inferences:

In [31]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

predictor = RealTimePredictor(
    endpoint='predmain-sk-ll-endpoint',
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_JSON)

payload = 'TID008,VAWT,64,80,46.0,21,55,55,7,34,NE'
print(predictor.predict(payload))

b'{"predictions": [{"score": 0.6406679749488831, "predicted_label": 1.0}]}'


<h2>Cleanup</h2>

Once finished testing the endpoint and getting some inferences, we can delete the endopoint to avoid incurring in unexpected charges.

In [25]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: predmain-sk-ll-endpoint
INFO:sagemaker:Deleting endpoint with name: predmain-sk-ll-endpoint
